In [ ]:
import networkx as nx
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import pm4py as pm4
from pm4py.algo.evaluation.generalization import algorithm as generalization_evaluator
import pandas as pd
from statistics import mean

##### Generate subgroup logs

In [ ]:
elog_data_first = pd.read_csv('data/export/cv19_event_log_wv1.csv')
elog_data_second = pd.read_csv('data/export/cv19_event_log_wv2.csv')
elog_data_full = pd.read_csv('data/export/cv19_event_log_full.csv')
elog_data_first['start'] = pd.to_datetime(elog_data_first['start'])
elog_data_second['start'] = pd.to_datetime(elog_data_second['start'])

In [ ]:
base_cohort = pd.read_csv('data/export/cv19_cohort.csv')

In [ ]:
el_full_ooh = elog_data_full.groupby('pid')['ooh'].count().reset_index()
el_full_ooh.ooh.quantile(0.8)

In [ ]:
elog_data_first.SIMD.value_counts()

In [ ]:
base_cohort.groupby('wave').n_morbid.value_counts()

In [ ]:
elog_data_first.columns

In [ ]:
el_ooh_1 = elog_data_first.groupby('pid')['ooh'].count().reset_index().rename(columns={'ooh': 'ooh_count'})
el_ooh_2 = elog_data_second.groupby('pid')['ooh'].count().reset_index().rename(columns={'ooh': 'ooh_count'})
elog_data_first = pd.merge(elog_data_first, el_ooh_1[['pid', 'ooh_count']], how='left', on='pid')
elog_data_second = pd.merge(elog_data_second, el_ooh_2[['pid', 'ooh_count']], how='left', on='pid')

In [ ]:
results_df = pd.DataFrame(columns=['subset', 'level', 'L1W2_lf', 'L1W2_prec', 'L1W2_gen', 'L2W1_lf', 'L2W1_prec', 'L2W1_gen', 'm_lf',
                          'm_prec', 'm_gen', 'GED', 'L1W2_percfit', 'L2W1_percfit', 'm_percfit'])

In [ ]:
elog_data_fp = elog_data_first.rename(columns={"pid": "case:concept:name", "provider_event": "concept:name",
                                            "start": "time:timestamp"})
elog_data_fp = pm4.format_dataframe(elog_data_fp, case_id='case:concept:name', activity_key='concept:name', timestamp_key='time:timestamp',
                                   timest_format='%Y-%m-%d %H:%M:%S')
elog_data_sp = elog_data_second.rename(columns={"pid": "case:concept:name", "provider_event": "concept:name",
                                            "start": "time:timestamp"})
elog_data_sp = pm4.format_dataframe(elog_data_sp, case_id='case:concept:name', activity_key='concept:name', timestamp_key='time:timestamp',
                                   timest_format='%Y-%m-%d %H:%M:%S')
elog_data_fa = elog_data_first.rename(columns={"pid": "case:concept:name", "act_code": "concept:name",
                                            "start": "time:timestamp"})
elog_data_fa = pm4.format_dataframe(elog_data_fa, case_id='case:concept:name', activity_key='concept:name', timestamp_key='time:timestamp',
                                   timest_format='%Y-%m-%d %H:%M:%S')
elog_data_sa = elog_data_second.rename(columns={"pid": "case:concept:name", "act_code": "concept:name",
                                            "start": "time:timestamp"})
elog_data_sa = pm4.format_dataframe(elog_data_sa, case_id='case:concept:name', activity_key='concept:name', timestamp_key='time:timestamp',
                                   timest_format='%Y-%m-%d %H:%M:%S')

#### Subsets

In [ ]:
### Over 75
el_data_fp_a75 = elog_data_fp[elog_data_fp.above_75 == 1]
el_data_sp_a75 = elog_data_sp[elog_data_sp.above_75 == 1]
el_data_fa_a75 = elog_data_fa[elog_data_fa.above_75 == 1]
el_data_sa_a75 = elog_data_sa[elog_data_sa.above_75 == 1]
### Intensive therapy
el_data_fp_int = elog_data_fp[elog_data_fp.spec == 'Intensive Therapy']
el_data_sp_int = elog_data_sp[elog_data_sp.spec == 'Intensive Therapy']
el_data_fa_int = elog_data_fa[elog_data_fa.spec == 'Intensive Therapy']
el_data_sa_int = elog_data_sa[elog_data_sa.spec == 'Intensive Therapy']
### Multimorbidity
el_data_fp_mm = elog_data_fp[elog_data_fp.o_mm == 1]
el_data_sp_mm = elog_data_sp[elog_data_sp.o_mm == 1]
el_data_fa_mm = elog_data_fa[elog_data_fa.o_mm == 1]
el_data_sa_mm = elog_data_sa[elog_data_sa.o_mm == 1]
### 1-year all-cause mortality
el_data_fp_m = elog_data_fp[elog_data_fp.o_1ym == 1]
el_data_sp_m = elog_data_sp[elog_data_sp.o_1ym == 1]
el_data_fa_m = elog_data_fa[elog_data_fa.o_1ym == 1]
el_data_sa_m = elog_data_sa[elog_data_sa.o_1ym == 1]
### Extended stay (>30 days)
el_data_fp_es = elog_data_fp[elog_data_fp.o_es == 1]
el_data_sp_es = elog_data_sp[elog_data_sp.o_es == 1]
el_data_fa_es = elog_data_fa[elog_data_fa.o_es == 1]
el_data_sa_es = elog_data_sa[elog_data_sa.o_es == 1]
### High out-of-hours care (>80th percentile of overall log)
el_data_fp_ooh = elog_data_fp[elog_data_fp.ooh_count > 70.0]
el_data_sp_ooh = elog_data_sp[elog_data_sp.ooh_count > 70.0]
el_data_fa_ooh = elog_data_fa[elog_data_fa.ooh_count > 70.0]
el_data_sa_ooh = elog_data_sa[elog_data_sa.ooh_count > 70.0]

In [ ]:
print(el_data_fp_a75.shape, el_data_fp_int.shape, el_data_fp_mm.shape, el_data_fp_m.shape, el_data_fp_es.shape, el_data_fp_ooh.shape)
print(el_data_sp_a75.shape, el_data_sp_int.shape, el_data_sp_mm.shape, el_data_sp_m.shape, el_data_sp_es.shape, el_data_sp_ooh.shape)
print(el_data_fa_a75.shape, el_data_fa_int.shape, el_data_fa_mm.shape, el_data_fa_m.shape, el_data_fa_es.shape, el_data_fa_ooh.shape)
print(el_data_sa_a75.shape, el_data_sa_int.shape, el_data_sa_mm.shape, el_data_sa_m.shape, el_data_sa_es.shape, el_data_sa_ooh.shape)

In [ ]:
el_fp_a75 = pm4.convert_to_event_log(el_data_fp_a75)
el_sp_a75 = pm4.convert_to_event_log(el_data_sp_a75)
el_fa_a75 = pm4.convert_to_event_log(el_data_fa_a75)
el_sa_a75 = pm4.convert_to_event_log(el_data_sa_a75)

el_fp_int = pm4.convert_to_event_log(el_data_fp_int)
el_sp_int = pm4.convert_to_event_log(el_data_sp_int)
el_fa_int = pm4.convert_to_event_log(el_data_fa_int)
el_sa_int = pm4.convert_to_event_log(el_data_sa_int)

el_fp_mm = pm4.convert_to_event_log(el_data_fp_mm)
el_sp_mm = pm4.convert_to_event_log(el_data_sp_mm)
el_fa_mm = pm4.convert_to_event_log(el_data_fa_mm)
el_sa_mm = pm4.convert_to_event_log(el_data_sa_mm)

el_fp_m = pm4.convert_to_event_log(el_data_fp_m)
el_sp_m = pm4.convert_to_event_log(el_data_sp_m)
el_fa_m = pm4.convert_to_event_log(el_data_fa_m)
el_sa_m = pm4.convert_to_event_log(el_data_sa_m)

el_fp_es = pm4.convert_to_event_log(el_data_fp_es)
el_sp_es = pm4.convert_to_event_log(el_data_sp_es)
el_fa_es = pm4.convert_to_event_log(el_data_fa_es)
el_sa_es = pm4.convert_to_event_log(el_data_sa_es)

el_fp_ooh = pm4.convert_to_event_log(el_data_fp_ooh)
el_sp_ooh = pm4.convert_to_event_log(el_data_sp_ooh)
el_fa_ooh = pm4.convert_to_event_log(el_data_fa_ooh)
el_sa_ooh = pm4.convert_to_event_log(el_data_sa_ooh)

##### Discover and save Petri Nets with Inductive Miner

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_fp_a75, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_fp_a75.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_sp_a75, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_sp_a75.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_fa_a75, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_fa_a75.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_sa_a75, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_sa_a75.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_fp_int, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_fp_int.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_sp_int, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_sp_int.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_fa_int, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_fa_int.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_sa_int, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_sa_int.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_fp_es, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_fp_es.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_sp_es, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_sp_es.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_fa_es, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_fa_es.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_sa_es, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_sa_es.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_fp_mm, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_fp_mm.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_fa_mm, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_fa_mm.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_sp_mm, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_sp_mm.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_sa_mm, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_sa_mm.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_fp_m, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_fp_m.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_sp_m, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_sp_m.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_fa_m, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_fa_m.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_sa_m, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_sa_m.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_fp_ooh, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_fp_ooh.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_fa_ooh, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_fa_ooh.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_sp_ooh, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_sp_ooh.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_sa_ooh, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_sa_ooh.pnml')

#### Cross-log conformance checking

In [ ]:
net1, im1, fm1 = pm4.read_pnml('process_mining/petri_nets/el_fp_a75.pnml')
net2, im2, fm2 = pm4.read_pnml('process_mining/petri_nets/el_sp_a75.pnml')

In [ ]:
fitness_L1W2 = pm4.fitness_token_based_replay(el_fp_a75, net2, im2, fm2)
prec_L1W2 = pm4.precision_token_based_replay(el_fp_a75, net2, im2, fm2)
gen_L1W2 = generalization_evaluator.apply(el_fp_a75, net2, im2, fm2)

fitness_L2W1 = pm4.fitness_token_based_replay(el_sp_a75, net1, im1, fm1)
prec_L2W1 = pm4.precision_token_based_replay(el_sp_a75, net1, im1, fm1)
gen_L2W1 = generalization_evaluator.apply(el_sp_a75, net1, im1, fm1)

In [ ]:
nx_pn = pm4.convert_petri_net_to_networkx(net1, im1, fm1)
nx_pn2 = pm4.convert_petri_net_to_networkx(net2, im2, fm2)
ged = nx.graph_edit_distance(nx_pn, nx_pn2, timeout=30)

In [ ]:
fitness_L2W1

In [ ]:
res_l = ['>75 years', 'Provider', round(fitness_L1W2['log_fitness'], 3), round(prec_L1W2, 3), round(gen_L1W2, 3),
         round(fitness_L2W1['log_fitness'], 3), round(prec_L2W1, 3), round(gen_L2W1, 3),
         round(mean([fitness_L1W2['log_fitness'], fitness_L2W1['log_fitness']]), 3),
         round(mean([prec_L1W2, prec_L2W1]), 3),
         round(mean([gen_L1W2, gen_L2W1]), 3),
        int(ged),
        round(fitness_L1W2['perc_fit_traces'], 3),
        round(fitness_L2W1['perc_fit_traces'], 3),
        round(mean([fitness_L1W2['perc_fit_traces'], fitness_L2W1['perc_fit_traces']]), 3)]
results_df.loc[len(results_df)] = res_l

In [ ]:
results_df

In [ ]:
net1, im1, fm1 = pm4.read_pnml('process_mining/petri_nets/el_fa_a75.pnml')
net2, im2, fm2 = pm4.read_pnml('process_mining/petri_nets/el_sa_a75.pnml')

fitness_L1W2 = pm4.fitness_token_based_replay(el_fa_a75, net2, im2, fm2)
prec_L1W2 = pm4.precision_token_based_replay(el_fa_a75, net2, im2, fm2)
gen_L1W2 = generalization_evaluator.apply(el_fa_a75, net2, im2, fm2)

fitness_L2W1 = pm4.fitness_token_based_replay(el_sa_a75, net1, im1, fm1)
prec_L2W1 = pm4.precision_token_based_replay(el_sa_a75, net1, im1, fm1)
gen_L2W1 = generalization_evaluator.apply(el_sa_a75, net1, im1, fm1)

nx_pn = pm4.convert_petri_net_to_networkx(net1, im1, fm1)
nx_pn2 = pm4.convert_petri_net_to_networkx(net2, im2, fm2)
ged = nx.graph_edit_distance(nx_pn, nx_pn2, timeout=30)

res_l = ['>75 years', 'Activity', round(fitness_L1W2['log_fitness'], 3), round(prec_L1W2, 3), round(gen_L1W2, 3),
         round(fitness_L2W1['log_fitness'], 3), round(prec_L2W1, 3), round(gen_L2W1, 3),
         round(mean([fitness_L1W2['log_fitness'], fitness_L2W1['log_fitness']]), 3),
         round(mean([prec_L1W2, prec_L2W1]), 3),
         round(mean([gen_L1W2, gen_L2W1]), 3),
        int(ged),
        round(fitness_L1W2['perc_fit_traces'], 3),
        round(fitness_L2W1['perc_fit_traces'], 3),
        round(mean([fitness_L1W2['perc_fit_traces'], fitness_L2W1['perc_fit_traces']]), 3)]
results_df.loc[len(results_df)] = res_l

In [ ]:
results_df

In [ ]:
net1, im1, fm1 = pm4.read_pnml('process_mining/petri_nets/el_fp_int.pnml')
net2, im2, fm2 = pm4.read_pnml('process_mining/petri_nets/el_sp_int.pnml')

fitness_L1W2 = pm4.fitness_token_based_replay(el_fp_int, net2, im2, fm2)
prec_L1W2 = pm4.precision_token_based_replay(el_fp_int, net2, im2, fm2)
gen_L1W2 = generalization_evaluator.apply(el_fp_int, net2, im2, fm2)

fitness_L2W1 = pm4.fitness_token_based_replay(el_sp_int, net1, im1, fm1)
prec_L2W1 = pm4.precision_token_based_replay(el_sp_int, net1, im1, fm1)
gen_L2W1 = generalization_evaluator.apply(el_sp_int, net1, im1, fm1)

nx_pn = pm4.convert_petri_net_to_networkx(net1, im1, fm1)
nx_pn2 = pm4.convert_petri_net_to_networkx(net2, im2, fm2)
ged = nx.graph_edit_distance(nx_pn, nx_pn2, timeout=30)

res_l = ['Intensive Therapy', 'Provider', round(fitness_L1W2['log_fitness'], 3), round(prec_L1W2, 3), round(gen_L1W2, 3),
         round(fitness_L2W1['log_fitness'], 3), round(prec_L2W1, 3), round(gen_L2W1, 3),
         round(mean([fitness_L1W2['log_fitness'], fitness_L2W1['log_fitness']]), 3),
         round(mean([prec_L1W2, prec_L2W1]), 3),
         round(mean([gen_L1W2, gen_L2W1]), 3),
        int(ged),
        round(fitness_L1W2['perc_fit_traces'], 3),
        round(fitness_L2W1['perc_fit_traces'], 3),
        round(mean([fitness_L1W2['perc_fit_traces'], fitness_L2W1['perc_fit_traces']]), 3)]
results_df.loc[len(results_df)] = res_l

In [ ]:
results_df

In [ ]:
net1, im1, fm1 = pm4.read_pnml('process_mining/petri_nets/el_fa_int.pnml')
net2, im2, fm2 = pm4.read_pnml('process_mining/petri_nets/el_sa_int.pnml')

fitness_L1W2 = pm4.fitness_token_based_replay(el_fa_int, net2, im2, fm2)
prec_L1W2 = pm4.precision_token_based_replay(el_fa_int, net2, im2, fm2)
gen_L1W2 = generalization_evaluator.apply(el_fa_int, net2, im2, fm2)

fitness_L2W1 = pm4.fitness_token_based_replay(el_sa_int, net1, im1, fm1)
prec_L2W1 = pm4.precision_token_based_replay(el_sa_int, net1, im1, fm1)
gen_L2W1 = generalization_evaluator.apply(el_sa_int, net1, im1, fm1)

nx_pn = pm4.convert_petri_net_to_networkx(net1, im1, fm1)
nx_pn2 = pm4.convert_petri_net_to_networkx(net2, im2, fm2)
ged = nx.graph_edit_distance(nx_pn, nx_pn2, timeout=30)

res_l = ['Intensive Therapy', 'Activity', round(fitness_L1W2['log_fitness'], 3), round(prec_L1W2, 3), round(gen_L1W2, 3),
         round(fitness_L2W1['log_fitness'], 3), round(prec_L2W1, 3), round(gen_L2W1, 3),
         round(mean([fitness_L1W2['log_fitness'], fitness_L2W1['log_fitness']]), 3),
         round(mean([prec_L1W2, prec_L2W1]), 3),
         round(mean([gen_L1W2, gen_L2W1]), 3),
        int(ged),
        round(fitness_L1W2['perc_fit_traces'], 3),
        round(fitness_L2W1['perc_fit_traces'], 3),
        round(mean([fitness_L1W2['perc_fit_traces'], fitness_L2W1['perc_fit_traces']]), 3)]
results_df.loc[len(results_df)] = res_l

In [ ]:
results_df

In [ ]:
net1, im1, fm1 = pm4.read_pnml('process_mining/petri_nets/el_fp_es.pnml')
net2, im2, fm2 = pm4.read_pnml('process_mining/petri_nets/el_sp_es.pnml')

fitness_L1W2 = pm4.fitness_token_based_replay(el_fp_es, net2, im2, fm2)
prec_L1W2 = pm4.precision_token_based_replay(el_fp_es, net2, im2, fm2)
gen_L1W2 = generalization_evaluator.apply(el_fp_es, net2, im2, fm2)

fitness_L2W1 = pm4.fitness_token_based_replay(el_sp_es, net1, im1, fm1)
prec_L2W1 = pm4.precision_token_based_replay(el_sp_es, net1, im1, fm1)
gen_L2W1 = generalization_evaluator.apply(el_sp_es, net1, im1, fm1)

nx_pn = pm4.convert_petri_net_to_networkx(net1, im1, fm1)
nx_pn2 = pm4.convert_petri_net_to_networkx(net2, im2, fm2)
ged = nx.graph_edit_distance(nx_pn, nx_pn2, timeout=30)

res_l = ['Extended Stay (>30 days)', 'Provider', round(fitness_L1W2['log_fitness'], 3), round(prec_L1W2, 3), round(gen_L1W2, 3),
         round(fitness_L2W1['log_fitness'], 3), round(prec_L2W1, 3), round(gen_L2W1, 3),
         round(mean([fitness_L1W2['log_fitness'], fitness_L2W1['log_fitness']]), 3),
         round(mean([prec_L1W2, prec_L2W1]), 3),
         round(mean([gen_L1W2, gen_L2W1]), 3),
        int(ged),
        round(fitness_L1W2['perc_fit_traces'], 3),
        round(fitness_L2W1['perc_fit_traces'], 3),
        round(mean([fitness_L1W2['perc_fit_traces'], fitness_L2W1['perc_fit_traces']]), 3)]
results_df.loc[len(results_df)] = res_l

In [ ]:
net1, im1, fm1 = pm4.read_pnml('process_mining/petri_nets/el_fa_es.pnml')
net2, im2, fm2 = pm4.read_pnml('process_mining/petri_nets/el_sa_es.pnml')

fitness_L1W2 = pm4.fitness_token_based_replay(el_fa_es, net2, im2, fm2)
prec_L1W2 = pm4.precision_token_based_replay(el_fa_es, net2, im2, fm2)
gen_L1W2 = generalization_evaluator.apply(el_fa_es, net2, im2, fm2)

fitness_L2W1 = pm4.fitness_token_based_replay(el_sa_es, net1, im1, fm1)
prec_L2W1 = pm4.precision_token_based_replay(el_sa_es, net1, im1, fm1)
gen_L2W1 = generalization_evaluator.apply(el_sa_es, net1, im1, fm1)

nx_pn = pm4.convert_petri_net_to_networkx(net1, im1, fm1)
nx_pn2 = pm4.convert_petri_net_to_networkx(net2, im2, fm2)
ged = nx.graph_edit_distance(nx_pn, nx_pn2, timeout=30)

res_l = ['Extended Stay (>30 days)', 'Activity', round(fitness_L1W2['log_fitness'], 3), round(prec_L1W2, 3), round(gen_L1W2, 3),
         round(fitness_L2W1['log_fitness'], 3), round(prec_L2W1, 3), round(gen_L2W1, 3),
         round(mean([fitness_L1W2['log_fitness'], fitness_L2W1['log_fitness']]), 3),
         round(mean([prec_L1W2, prec_L2W1]), 3),
         round(mean([gen_L1W2, gen_L2W1]), 3),
        int(ged),
        round(fitness_L1W2['perc_fit_traces'], 3),
        round(fitness_L2W1['perc_fit_traces'], 3),
        round(mean([fitness_L1W2['perc_fit_traces'], fitness_L2W1['perc_fit_traces']]), 3)]
results_df.loc[len(results_df)] = res_l

In [ ]:
net1, im1, fm1 = pm4.read_pnml('process_mining/petri_nets/el_fp_m.pnml')
net2, im2, fm2 = pm4.read_pnml('process_mining/petri_nets/el_sp_m.pnml')

fitness_L1W2 = pm4.fitness_token_based_replay(el_fp_m, net2, im2, fm2)
prec_L1W2 = pm4.precision_token_based_replay(el_fp_m, net2, im2, fm2)
gen_L1W2 = generalization_evaluator.apply(el_fp_m, net2, im2, fm2)

fitness_L2W1 = pm4.fitness_token_based_replay(el_sp_m, net1, im1, fm1)
prec_L2W1 = pm4.precision_token_based_replay(el_sp_m, net1, im1, fm1)
gen_L2W1 = generalization_evaluator.apply(el_sp_m, net1, im1, fm1)

nx_pn = pm4.convert_petri_net_to_networkx(net1, im1, fm1)
nx_pn2 = pm4.convert_petri_net_to_networkx(net2, im2, fm2)
ged = nx.graph_edit_distance(nx_pn, nx_pn2, timeout=30)

res_l = ['1-year mortality', 'Provider', round(fitness_L1W2['log_fitness'], 3), round(prec_L1W2, 3), round(gen_L1W2, 3),
         round(fitness_L2W1['log_fitness'], 3), round(prec_L2W1, 3), round(gen_L2W1, 3),
         round(mean([fitness_L1W2['log_fitness'], fitness_L2W1['log_fitness']]), 3),
         round(mean([prec_L1W2, prec_L2W1]), 3),
         round(mean([gen_L1W2, gen_L2W1]), 3),
        int(ged),
        round(fitness_L1W2['perc_fit_traces'], 3),
        round(fitness_L2W1['perc_fit_traces'], 3),
        round(mean([fitness_L1W2['perc_fit_traces'], fitness_L2W1['perc_fit_traces']]), 3)]
results_df.loc[len(results_df)] = res_l

In [ ]:
net1, im1, fm1 = pm4.read_pnml('process_mining/petri_nets/el_fa_m.pnml')
net2, im2, fm2 = pm4.read_pnml('process_mining/petri_nets/el_sa_m.pnml')

fitness_L1W2 = pm4.fitness_token_based_replay(el_fa_m, net2, im2, fm2)
prec_L1W2 = pm4.precision_token_based_replay(el_fa_m, net2, im2, fm2)
gen_L1W2 = generalization_evaluator.apply(el_fa_m, net2, im2, fm2)

fitness_L2W1 = pm4.fitness_token_based_replay(el_sa_m, net1, im1, fm1)
prec_L2W1 = pm4.precision_token_based_replay(el_sa_m, net1, im1, fm1)
gen_L2W1 = generalization_evaluator.apply(el_sa_m, net1, im1, fm1)

nx_pn = pm4.convert_petri_net_to_networkx(net1, im1, fm1)
nx_pn2 = pm4.convert_petri_net_to_networkx(net2, im2, fm2)
ged = nx.graph_edit_distance(nx_pn, nx_pn2, timeout=30)

res_l = ['1-year mortality', 'Activity', round(fitness_L1W2['log_fitness'], 3), round(prec_L1W2, 3), round(gen_L1W2, 3),
         round(fitness_L2W1['log_fitness'], 3), round(prec_L2W1, 3), round(gen_L2W1, 3),
         round(mean([fitness_L1W2['log_fitness'], fitness_L2W1['log_fitness']]), 3),
         round(mean([prec_L1W2, prec_L2W1]), 3),
         round(mean([gen_L1W2, gen_L2W1]), 3),
        int(ged),
        round(fitness_L1W2['perc_fit_traces'], 3),
        round(fitness_L2W1['perc_fit_traces'], 3),
        round(mean([fitness_L1W2['perc_fit_traces'], fitness_L2W1['perc_fit_traces']]), 3)]
results_df.loc[len(results_df)] = res_l

In [ ]:
net1, im1, fm1 = pm4.read_pnml('process_mining/petri_nets/el_fp_mm.pnml')
net2, im2, fm2 = pm4.read_pnml('process_mining/petri_nets/el_sp_mm.pnml')

fitness_L1W2 = pm4.fitness_token_based_replay(el_fp_mm, net2, im2, fm2)
prec_L1W2 = pm4.precision_token_based_replay(el_fp_mm, net2, im2, fm2)
gen_L1W2 = generalization_evaluator.apply(el_fp_mm, net2, im2, fm2)

fitness_L2W1 = pm4.fitness_token_based_replay(el_sp_mm, net1, im1, fm1)
prec_L2W1 = pm4.precision_token_based_replay(el_sp_mm, net1, im1, fm1)
gen_L2W1 = generalization_evaluator.apply(el_sp_mm, net1, im1, fm1)

nx_pn = pm4.convert_petri_net_to_networkx(net1, im1, fm1)
nx_pn2 = pm4.convert_petri_net_to_networkx(net2, im2, fm2)
ged = nx.graph_edit_distance(nx_pn, nx_pn2, timeout=30)

res_l = ['Multimorbidity', 'Provider', round(fitness_L1W2['log_fitness'], 3), round(prec_L1W2, 3), round(gen_L1W2, 3),
         round(fitness_L2W1['log_fitness'], 3), round(prec_L2W1, 3), round(gen_L2W1, 3),
         round(mean([fitness_L1W2['log_fitness'], fitness_L2W1['log_fitness']]), 3),
         round(mean([prec_L1W2, prec_L2W1]), 3),
         round(mean([gen_L1W2, gen_L2W1]), 3),
        int(ged),
        round(fitness_L1W2['perc_fit_traces'], 3),
        round(fitness_L2W1['perc_fit_traces'], 3),
        round(mean([fitness_L1W2['perc_fit_traces'], fitness_L2W1['perc_fit_traces']]), 3)]
results_df.loc[len(results_df)] = res_l

In [ ]:
net1, im1, fm1 = pm4.read_pnml('process_mining/petri_nets/el_fa_mm.pnml')
net2, im2, fm2 = pm4.read_pnml('process_mining/petri_nets/el_sa_mm.pnml')

fitness_L1W2 = pm4.fitness_token_based_replay(el_fa_mm, net2, im2, fm2)
prec_L1W2 = pm4.precision_token_based_replay(el_fa_mm, net2, im2, fm2)
gen_L1W2 = generalization_evaluator.apply(el_fa_mm, net2, im2, fm2)

fitness_L2W1 = pm4.fitness_token_based_replay(el_sa_mm, net1, im1, fm1)
prec_L2W1 = pm4.precision_token_based_replay(el_sa_mm, net1, im1, fm1)
gen_L2W1 = generalization_evaluator.apply(el_sa_mm, net1, im1, fm1)

nx_pn = pm4.convert_petri_net_to_networkx(net1, im1, fm1)
nx_pn2 = pm4.convert_petri_net_to_networkx(net2, im2, fm2)
ged = nx.graph_edit_distance(nx_pn, nx_pn2, timeout=30)

res_l = ['Multimorbidity', 'Activity', round(fitness_L1W2['log_fitness'], 3), round(prec_L1W2, 3), round(gen_L1W2, 3),
         round(fitness_L2W1['log_fitness'], 3), round(prec_L2W1, 3), round(gen_L2W1, 3),
         round(mean([fitness_L1W2['log_fitness'], fitness_L2W1['log_fitness']]), 3),
         round(mean([prec_L1W2, prec_L2W1]), 3),
         round(mean([gen_L1W2, gen_L2W1]), 3),
        int(ged),
        round(fitness_L1W2['perc_fit_traces'], 3),
        round(fitness_L2W1['perc_fit_traces'], 3),
        round(mean([fitness_L1W2['perc_fit_traces'], fitness_L2W1['perc_fit_traces']]), 3)]
results_df.loc[len(results_df)] = res_l

In [ ]:
net1, im1, fm1 = pm4.read_pnml('process_mining/petri_nets/el_fp_ooh.pnml')
net2, im2, fm2 = pm4.read_pnml('process_mining/petri_nets/el_sp_ooh.pnml')

fitness_L1W2 = pm4.fitness_token_based_replay(el_fp_ooh, net2, im2, fm2)
prec_L1W2 = pm4.precision_token_based_replay(el_fp_ooh, net2, im2, fm2)
gen_L1W2 = generalization_evaluator.apply(el_fp_ooh, net2, im2, fm2)

fitness_L2W1 = pm4.fitness_token_based_replay(el_sp_ooh, net1, im1, fm1)
prec_L2W1 = pm4.precision_token_based_replay(el_sp_ooh, net1, im1, fm1)
gen_L2W1 = generalization_evaluator.apply(el_sp_ooh, net1, im1, fm1)

nx_pn = pm4.convert_petri_net_to_networkx(net1, im1, fm1)
nx_pn2 = pm4.convert_petri_net_to_networkx(net2, im2, fm2)
ged = nx.graph_edit_distance(nx_pn, nx_pn2, timeout=30)

res_l = ['High Out-of-hours care (top 20% overall)', 'Provider', round(fitness_L1W2['log_fitness'], 3), round(prec_L1W2, 3), round(gen_L1W2, 3),
         round(fitness_L2W1['log_fitness'], 3), round(prec_L2W1, 3), round(gen_L2W1, 3),
         round(mean([fitness_L1W2['log_fitness'], fitness_L2W1['log_fitness']]), 3),
         round(mean([prec_L1W2, prec_L2W1]), 3),
         round(mean([gen_L1W2, gen_L2W1]), 3),
        int(ged),
        round(fitness_L1W2['perc_fit_traces'], 3),
        round(fitness_L2W1['perc_fit_traces'], 3),
        round(mean([fitness_L1W2['perc_fit_traces'], fitness_L2W1['perc_fit_traces']]), 3)]
results_df.loc[len(results_df)] = res_l

In [ ]:
net1, im1, fm1 = pm4.read_pnml('process_mining/petri_nets/el_fa_ooh.pnml')
net2, im2, fm2 = pm4.read_pnml('process_mining/petri_nets/el_sa_ooh.pnml')

fitness_L1W2 = pm4.fitness_token_based_replay(el_fa_ooh, net2, im2, fm2)
prec_L1W2 = pm4.precision_token_based_replay(el_fa_ooh, net2, im2, fm2)
gen_L1W2 = generalization_evaluator.apply(el_fa_ooh, net2, im2, fm2)

fitness_L2W1 = pm4.fitness_token_based_replay(el_sa_ooh, net1, im1, fm1)
prec_L2W1 = pm4.precision_token_based_replay(el_sa_ooh, net1, im1, fm1)
gen_L2W1 = generalization_evaluator.apply(el_sa_ooh, net1, im1, fm1)

nx_pn = pm4.convert_petri_net_to_networkx(net1, im1, fm1)
nx_pn2 = pm4.convert_petri_net_to_networkx(net2, im2, fm2)
ged = nx.graph_edit_distance(nx_pn, nx_pn2, timeout=30)

res_l = ['High Out-of-hours care (top 20% overall)', 'Activity', round(fitness_L1W2['log_fitness'], 3), round(prec_L1W2, 3), round(gen_L1W2, 3),
         round(fitness_L2W1['log_fitness'], 3), round(prec_L2W1, 3), round(gen_L2W1, 3),
         round(mean([fitness_L1W2['log_fitness'], fitness_L2W1['log_fitness']]), 3),
         round(mean([prec_L1W2, prec_L2W1]), 3),
         round(mean([gen_L1W2, gen_L2W1]), 3),
        int(ged),
        round(fitness_L1W2['perc_fit_traces'], 3),
        round(fitness_L2W1['perc_fit_traces'], 3),
        round(mean([fitness_L1W2['perc_fit_traces'], fitness_L2W1['perc_fit_traces']]), 3)]
results_df.loc[len(results_df)] = res_l

In [ ]:
results_df

#### Re-run for overall models

In [ ]:
el_fp = pm4.convert_to_event_log(elog_data_fp)
el_sp = pm4.convert_to_event_log(elog_data_sp)
el_fa = pm4.convert_to_event_log(elog_data_fa)
el_sa = pm4.convert_to_event_log(elog_data_sa)

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_fp, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_fp.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_fa, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_fa.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_sp, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_sp.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_sa, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_sa.pnml')

In [ ]:
results_df.to_csv('process_mining/crosslog_cc_results.csv', index=False)

##### Append negative group results

In [ ]:
res_df = pd.read_csv('process_mining/crosslog_cc_results.csv')
res_df

In [ ]:
### Under 75
el_data_fp_a75 = elog_data_fp[elog_data_fp.above_75 == 0]
el_data_sp_a75 = elog_data_sp[elog_data_sp.above_75 == 0]
el_data_fa_a75 = elog_data_fa[elog_data_fa.above_75 == 0]
el_data_sa_a75 = elog_data_sa[elog_data_sa.above_75 == 0]
### NOT Intensive therapy
el_data_fp_int = elog_data_fp[elog_data_fp.spec != 'Intensive Therapy']
el_data_sp_int = elog_data_sp[elog_data_sp.spec != 'Intensive Therapy']
el_data_fa_int = elog_data_fa[elog_data_fa.spec != 'Intensive Therapy']
el_data_sa_int = elog_data_sa[elog_data_sa.spec != 'Intensive Therapy']
### NOT Multimorbidity
el_data_fp_mm = elog_data_fp[elog_data_fp.o_mm == 0]
el_data_sp_mm = elog_data_sp[elog_data_sp.o_mm == 0]
el_data_fa_mm = elog_data_fa[elog_data_fa.o_mm == 0]
el_data_sa_mm = elog_data_sa[elog_data_sa.o_mm == 0]
### NOT 1-year all-cause mortality
el_data_fp_m = elog_data_fp[elog_data_fp.o_1ym == 0]
el_data_sp_m = elog_data_sp[elog_data_sp.o_1ym == 0]
el_data_fa_m = elog_data_fa[elog_data_fa.o_1ym == 0]
el_data_sa_m = elog_data_sa[elog_data_sa.o_1ym == 0]
### NOT Extended stay (>30 days)
el_data_fp_es = elog_data_fp[elog_data_fp.o_es == 0]
el_data_sp_es = elog_data_sp[elog_data_sp.o_es == 0]
el_data_fa_es = elog_data_fa[elog_data_fa.o_es == 0]
el_data_sa_es = elog_data_sa[elog_data_sa.o_es == 0]
### NOT High out-of-hours care (>80th percentile of overall log)
el_data_fp_ooh = elog_data_fp[elog_data_fp.ooh_count <= 70.0]
el_data_sp_ooh = elog_data_sp[elog_data_sp.ooh_count <= 70.0]
el_data_fa_ooh = elog_data_fa[elog_data_fa.ooh_count <= 70.0]
el_data_sa_ooh = elog_data_sa[elog_data_sa.ooh_count <= 70.0]

In [ ]:
print(el_data_fp_a75.shape, el_data_fp_int.shape, el_data_fp_mm.shape, el_data_fp_m.shape, el_data_fp_es.shape, el_data_fp_ooh.shape)
print(el_data_sp_a75.shape, el_data_sp_int.shape, el_data_sp_mm.shape, el_data_sp_m.shape, el_data_sp_es.shape, el_data_sp_ooh.shape)
print(el_data_fa_a75.shape, el_data_fa_int.shape, el_data_fa_mm.shape, el_data_fa_m.shape, el_data_fa_es.shape, el_data_fa_ooh.shape)
print(el_data_sa_a75.shape, el_data_sa_int.shape, el_data_sa_mm.shape, el_data_sa_m.shape, el_data_sa_es.shape, el_data_sa_ooh.shape)

In [ ]:
el_fp_a75 = pm4.convert_to_event_log(el_data_fp_a75)
el_sp_a75 = pm4.convert_to_event_log(el_data_sp_a75)
el_fa_a75 = pm4.convert_to_event_log(el_data_fa_a75)
el_sa_a75 = pm4.convert_to_event_log(el_data_sa_a75)

el_fp_int = pm4.convert_to_event_log(el_data_fp_int)
el_sp_int = pm4.convert_to_event_log(el_data_sp_int)
el_fa_int = pm4.convert_to_event_log(el_data_fa_int)
el_sa_int = pm4.convert_to_event_log(el_data_sa_int)

el_fp_mm = pm4.convert_to_event_log(el_data_fp_mm)
el_sp_mm = pm4.convert_to_event_log(el_data_sp_mm)
el_fa_mm = pm4.convert_to_event_log(el_data_fa_mm)
el_sa_mm = pm4.convert_to_event_log(el_data_sa_mm)

el_fp_m = pm4.convert_to_event_log(el_data_fp_m)
el_sp_m = pm4.convert_to_event_log(el_data_sp_m)
el_fa_m = pm4.convert_to_event_log(el_data_fa_m)
el_sa_m = pm4.convert_to_event_log(el_data_sa_m)

el_fp_es = pm4.convert_to_event_log(el_data_fp_es)
el_sp_es = pm4.convert_to_event_log(el_data_sp_es)
el_fa_es = pm4.convert_to_event_log(el_data_fa_es)
el_sa_es = pm4.convert_to_event_log(el_data_sa_es)

el_fp_ooh = pm4.convert_to_event_log(el_data_fp_ooh)
el_sp_ooh = pm4.convert_to_event_log(el_data_sp_ooh)
el_fa_ooh = pm4.convert_to_event_log(el_data_fa_ooh)
el_sa_ooh = pm4.convert_to_event_log(el_data_sa_ooh)

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_fp_a75, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_fp_aNOT75.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_sp_a75, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_sp_aNOT75.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_fa_a75, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_fa_aNOT75.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_sa_a75, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_sa_aNOT75.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_fp_int, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_fp_NOTint.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_sp_int, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_sp_NOTint.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_fa_int, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_fa_NOTint.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_sa_int, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_sa_NOTint.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_fp_es, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_fp_NOTes.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_sp_es, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_sp_NOTes.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_fa_es, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_fa_NOTes.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_sa_es, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_sa_NOTes.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_fp_mm, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_fp_NOTmm.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_sp_mm, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_sp_NOTmm.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_fa_mm, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_fa_NOTmm.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_sa_mm, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_sa_NOTmm.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_fp_m, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_fp_NOTm.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_sp_m, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_sp_NOTm.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_fa_m, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_fa_NOTm.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_sa_m, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_sa_NOTm.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_fp_ooh, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_fp_NOTooh.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_sp_ooh, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_sp_NOTooh.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_fa_ooh, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_fa_NOTooh.pnml')

In [ ]:
net, initial_marking, final_marking = pm4.discover_petri_net_inductive(el_sa_ooh, noise_threshold=0.3, multi_processing=True)
pm4.view_petri_net(net, initial_marking, final_marking)
pm4.write_pnml(net, initial_marking, final_marking, 'process_mining/petri_nets/el_sa_NOTooh.pnml')

#### Conformance check negative groups

In [ ]:
results_df

In [ ]:
net1, im1, fm1 = pm4.read_pnml('process_mining/petri_nets/el_fp_aNOT75.pnml')
net2, im2, fm2 = pm4.read_pnml('process_mining/petri_nets/el_sp_aNOT75.pnml')

fitness_L1W2 = pm4.fitness_token_based_replay(el_fp_a75, net2, im2, fm2)
prec_L1W2 = pm4.precision_token_based_replay(el_fp_a75, net2, im2, fm2)
gen_L1W2 = generalization_evaluator.apply(el_fp_a75, net2, im2, fm2)

fitness_L2W1 = pm4.fitness_token_based_replay(el_sp_a75, net1, im1, fm1)
prec_L2W1 = pm4.precision_token_based_replay(el_sp_a75, net1, im1, fm1)
gen_L2W1 = generalization_evaluator.apply(el_sp_a75, net1, im1, fm1)

nx_pn = pm4.convert_petri_net_to_networkx(net1, im1, fm1)
nx_pn2 = pm4.convert_petri_net_to_networkx(net2, im2, fm2)
ged = nx.graph_edit_distance(nx_pn, nx_pn2, timeout=30)

res_l = ['<75 years', 'Provider', round(fitness_L1W2['log_fitness'], 3), round(prec_L1W2, 3), round(gen_L1W2, 3),
         round(fitness_L2W1['log_fitness'], 3), round(prec_L2W1, 3), round(gen_L2W1, 3),
         round(mean([fitness_L1W2['log_fitness'], fitness_L2W1['log_fitness']]), 3),
         round(mean([prec_L1W2, prec_L2W1]), 3),
         round(mean([gen_L1W2, gen_L2W1]), 3),
        int(ged),
        round(fitness_L1W2['perc_fit_traces'], 3),
        round(fitness_L2W1['perc_fit_traces'], 3),
        round(mean([fitness_L1W2['perc_fit_traces'], fitness_L2W1['perc_fit_traces']]), 3)]
results_df.loc[len(results_df)] = res_l

In [ ]:
results_df

In [ ]:
net1, im1, fm1 = pm4.read_pnml('process_mining/petri_nets/el_fa_aNOT75.pnml')
net2, im2, fm2 = pm4.read_pnml('process_mining/petri_nets/el_sa_aNOT75.pnml')

fitness_L1W2 = pm4.fitness_token_based_replay(el_fa_a75, net2, im2, fm2)
prec_L1W2 = pm4.precision_token_based_replay(el_fa_a75, net2, im2, fm2)
gen_L1W2 = generalization_evaluator.apply(el_fa_a75, net2, im2, fm2)

fitness_L2W1 = pm4.fitness_token_based_replay(el_sa_a75, net1, im1, fm1)
prec_L2W1 = pm4.precision_token_based_replay(el_sa_a75, net1, im1, fm1)
gen_L2W1 = generalization_evaluator.apply(el_sa_a75, net1, im1, fm1)

nx_pn = pm4.convert_petri_net_to_networkx(net1, im1, fm1)
nx_pn2 = pm4.convert_petri_net_to_networkx(net2, im2, fm2)
ged = nx.graph_edit_distance(nx_pn, nx_pn2, timeout=30)

res_l = ['<75 years', 'Activity', round(fitness_L1W2['log_fitness'], 3), round(prec_L1W2, 3), round(gen_L1W2, 3),
         round(fitness_L2W1['log_fitness'], 3), round(prec_L2W1, 3), round(gen_L2W1, 3),
         round(mean([fitness_L1W2['log_fitness'], fitness_L2W1['log_fitness']]), 3),
         round(mean([prec_L1W2, prec_L2W1]), 3),
         round(mean([gen_L1W2, gen_L2W1]), 3),
        int(ged),
        round(fitness_L1W2['perc_fit_traces'], 3),
        round(fitness_L2W1['perc_fit_traces'], 3),
        round(mean([fitness_L1W2['perc_fit_traces'], fitness_L2W1['perc_fit_traces']]), 3)]
results_df.loc[len(results_df)] = res_l

In [ ]:
results_df

In [ ]:
net1, im1, fm1 = pm4.read_pnml('process_mining/petri_nets/el_fp_NOTint.pnml')
net2, im2, fm2 = pm4.read_pnml('process_mining/petri_nets/el_sp_NOTint.pnml')

fitness_L1W2 = pm4.fitness_token_based_replay(el_fp_int, net2, im2, fm2)
prec_L1W2 = pm4.precision_token_based_replay(el_fp_int, net2, im2, fm2)
gen_L1W2 = generalization_evaluator.apply(el_fp_int, net2, im2, fm2)

fitness_L2W1 = pm4.fitness_token_based_replay(el_sp_int, net1, im1, fm1)
prec_L2W1 = pm4.precision_token_based_replay(el_sp_int, net1, im1, fm1)
gen_L2W1 = generalization_evaluator.apply(el_sp_int, net1, im1, fm1)

nx_pn = pm4.convert_petri_net_to_networkx(net1, im1, fm1)
nx_pn2 = pm4.convert_petri_net_to_networkx(net2, im2, fm2)
ged = nx.graph_edit_distance(nx_pn, nx_pn2, timeout=30)

res_l = ['N_Intensive Therapy', 'Provider', round(fitness_L1W2['log_fitness'], 3), round(prec_L1W2, 3), round(gen_L1W2, 3),
         round(fitness_L2W1['log_fitness'], 3), round(prec_L2W1, 3), round(gen_L2W1, 3),
         round(mean([fitness_L1W2['log_fitness'], fitness_L2W1['log_fitness']]), 3),
         round(mean([prec_L1W2, prec_L2W1]), 3),
         round(mean([gen_L1W2, gen_L2W1]), 3),
        int(ged),
        round(fitness_L1W2['perc_fit_traces'], 3),
        round(fitness_L2W1['perc_fit_traces'], 3),
        round(mean([fitness_L1W2['perc_fit_traces'], fitness_L2W1['perc_fit_traces']]), 3)]
results_df.loc[len(results_df)] = res_l

In [ ]:
net1, im1, fm1 = pm4.read_pnml('process_mining/petri_nets/el_fa_NOTint.pnml')
net2, im2, fm2 = pm4.read_pnml('process_mining/petri_nets/el_sa_NOTint.pnml')

fitness_L1W2 = pm4.fitness_token_based_replay(el_fa_int, net2, im2, fm2)
prec_L1W2 = pm4.precision_token_based_replay(el_fa_int, net2, im2, fm2)
gen_L1W2 = generalization_evaluator.apply(el_fa_int, net2, im2, fm2)

fitness_L2W1 = pm4.fitness_token_based_replay(el_sa_int, net1, im1, fm1)
prec_L2W1 = pm4.precision_token_based_replay(el_sa_int, net1, im1, fm1)
gen_L2W1 = generalization_evaluator.apply(el_sa_int, net1, im1, fm1)

nx_pn = pm4.convert_petri_net_to_networkx(net1, im1, fm1)
nx_pn2 = pm4.convert_petri_net_to_networkx(net2, im2, fm2)
ged = nx.graph_edit_distance(nx_pn, nx_pn2, timeout=30)

res_l = ['N_Intensive Therapy', 'Activity', round(fitness_L1W2['log_fitness'], 3), round(prec_L1W2, 3), round(gen_L1W2, 3),
         round(fitness_L2W1['log_fitness'], 3), round(prec_L2W1, 3), round(gen_L2W1, 3),
         round(mean([fitness_L1W2['log_fitness'], fitness_L2W1['log_fitness']]), 3),
         round(mean([prec_L1W2, prec_L2W1]), 3),
         round(mean([gen_L1W2, gen_L2W1]), 3),
        int(ged),
        round(fitness_L1W2['perc_fit_traces'], 3),
        round(fitness_L2W1['perc_fit_traces'], 3),
        round(mean([fitness_L1W2['perc_fit_traces'], fitness_L2W1['perc_fit_traces']]), 3)]
results_df.loc[len(results_df)] = res_l

In [ ]:
net1, im1, fm1 = pm4.read_pnml('process_mining/petri_nets/el_fp_NOTes.pnml')
net2, im2, fm2 = pm4.read_pnml('process_mining/petri_nets/el_sp_NOTes.pnml')

fitness_L1W2 = pm4.fitness_token_based_replay(el_fp_es, net2, im2, fm2)
prec_L1W2 = pm4.precision_token_based_replay(el_fp_es, net2, im2, fm2)
gen_L1W2 = generalization_evaluator.apply(el_fp_es, net2, im2, fm2)

fitness_L2W1 = pm4.fitness_token_based_replay(el_sp_es, net1, im1, fm1)
prec_L2W1 = pm4.precision_token_based_replay(el_sp_es, net1, im1, fm1)
gen_L2W1 = generalization_evaluator.apply(el_sp_es, net1, im1, fm1)

nx_pn = pm4.convert_petri_net_to_networkx(net1, im1, fm1)
nx_pn2 = pm4.convert_petri_net_to_networkx(net2, im2, fm2)
ged = nx.graph_edit_distance(nx_pn, nx_pn2, timeout=30)

res_l = ['N_Extended Stay (>30 days)', 'Provider', round(fitness_L1W2['log_fitness'], 3), round(prec_L1W2, 3), round(gen_L1W2, 3),
         round(fitness_L2W1['log_fitness'], 3), round(prec_L2W1, 3), round(gen_L2W1, 3),
         round(mean([fitness_L1W2['log_fitness'], fitness_L2W1['log_fitness']]), 3),
         round(mean([prec_L1W2, prec_L2W1]), 3),
         round(mean([gen_L1W2, gen_L2W1]), 3),
        int(ged),
        round(fitness_L1W2['perc_fit_traces'], 3),
        round(fitness_L2W1['perc_fit_traces'], 3),
        round(mean([fitness_L1W2['perc_fit_traces'], fitness_L2W1['perc_fit_traces']]), 3)]
results_df.loc[len(results_df)] = res_l

In [ ]:
net1, im1, fm1 = pm4.read_pnml('process_mining/petri_nets/el_fa_NOTes.pnml')
net2, im2, fm2 = pm4.read_pnml('process_mining/petri_nets/el_sa_NOTes.pnml')

fitness_L1W2 = pm4.fitness_token_based_replay(el_fa_es, net2, im2, fm2)
prec_L1W2 = pm4.precision_token_based_replay(el_fa_es, net2, im2, fm2)
gen_L1W2 = generalization_evaluator.apply(el_fa_es, net2, im2, fm2)

fitness_L2W1 = pm4.fitness_token_based_replay(el_sa_es, net1, im1, fm1)
prec_L2W1 = pm4.precision_token_based_replay(el_sa_es, net1, im1, fm1)
gen_L2W1 = generalization_evaluator.apply(el_sa_es, net1, im1, fm1)

nx_pn = pm4.convert_petri_net_to_networkx(net1, im1, fm1)
nx_pn2 = pm4.convert_petri_net_to_networkx(net2, im2, fm2)
ged = nx.graph_edit_distance(nx_pn, nx_pn2, timeout=30)

res_l = ['N_Extended Stay (>30 days)', 'Activity', round(fitness_L1W2['log_fitness'], 3), round(prec_L1W2, 3), round(gen_L1W2, 3),
         round(fitness_L2W1['log_fitness'], 3), round(prec_L2W1, 3), round(gen_L2W1, 3),
         round(mean([fitness_L1W2['log_fitness'], fitness_L2W1['log_fitness']]), 3),
         round(mean([prec_L1W2, prec_L2W1]), 3),
         round(mean([gen_L1W2, gen_L2W1]), 3),
        int(ged),
        round(fitness_L1W2['perc_fit_traces'], 3),
        round(fitness_L2W1['perc_fit_traces'], 3),
        round(mean([fitness_L1W2['perc_fit_traces'], fitness_L2W1['perc_fit_traces']]), 3)]
results_df.loc[len(results_df)] = res_l

In [ ]:
net1, im1, fm1 = pm4.read_pnml('process_mining/petri_nets/el_fp_NOTm.pnml')
net2, im2, fm2 = pm4.read_pnml('process_mining/petri_nets/el_sp_NOTm.pnml')

fitness_L1W2 = pm4.fitness_token_based_replay(el_fp_m, net2, im2, fm2)
prec_L1W2 = pm4.precision_token_based_replay(el_fp_m, net2, im2, fm2)
gen_L1W2 = generalization_evaluator.apply(el_fp_m, net2, im2, fm2)

fitness_L2W1 = pm4.fitness_token_based_replay(el_sp_m, net1, im1, fm1)
prec_L2W1 = pm4.precision_token_based_replay(el_sp_m, net1, im1, fm1)
gen_L2W1 = generalization_evaluator.apply(el_sp_m, net1, im1, fm1)

nx_pn = pm4.convert_petri_net_to_networkx(net1, im1, fm1)
nx_pn2 = pm4.convert_petri_net_to_networkx(net2, im2, fm2)
ged = nx.graph_edit_distance(nx_pn, nx_pn2, timeout=30)

res_l = ['N_1-year mortality', 'Provider', round(fitness_L1W2['log_fitness'], 3), round(prec_L1W2, 3), round(gen_L1W2, 3),
         round(fitness_L2W1['log_fitness'], 3), round(prec_L2W1, 3), round(gen_L2W1, 3),
         round(mean([fitness_L1W2['log_fitness'], fitness_L2W1['log_fitness']]), 3),
         round(mean([prec_L1W2, prec_L2W1]), 3),
         round(mean([gen_L1W2, gen_L2W1]), 3),
        int(ged),
        round(fitness_L1W2['perc_fit_traces'], 3),
        round(fitness_L2W1['perc_fit_traces'], 3),
        round(mean([fitness_L1W2['perc_fit_traces'], fitness_L2W1['perc_fit_traces']]), 3)]
results_df.loc[len(results_df)] = res_l

In [ ]:
net1, im1, fm1 = pm4.read_pnml('process_mining/petri_nets/el_fa_NOTm.pnml')
net2, im2, fm2 = pm4.read_pnml('process_mining/petri_nets/el_sa_NOTm.pnml')

fitness_L1W2 = pm4.fitness_token_based_replay(el_fa_m, net2, im2, fm2)
prec_L1W2 = pm4.precision_token_based_replay(el_fa_m, net2, im2, fm2)
gen_L1W2 = generalization_evaluator.apply(el_fa_m, net2, im2, fm2)

fitness_L2W1 = pm4.fitness_token_based_replay(el_sa_m, net1, im1, fm1)
prec_L2W1 = pm4.precision_token_based_replay(el_sa_m, net1, im1, fm1)
gen_L2W1 = generalization_evaluator.apply(el_sa_m, net1, im1, fm1)

nx_pn = pm4.convert_petri_net_to_networkx(net1, im1, fm1)
nx_pn2 = pm4.convert_petri_net_to_networkx(net2, im2, fm2)
ged = nx.graph_edit_distance(nx_pn, nx_pn2, timeout=30)

res_l = ['N_1-year mortality', 'Activity', round(fitness_L1W2['log_fitness'], 3), round(prec_L1W2, 3), round(gen_L1W2, 3),
         round(fitness_L2W1['log_fitness'], 3), round(prec_L2W1, 3), round(gen_L2W1, 3),
         round(mean([fitness_L1W2['log_fitness'], fitness_L2W1['log_fitness']]), 3),
         round(mean([prec_L1W2, prec_L2W1]), 3),
         round(mean([gen_L1W2, gen_L2W1]), 3),
        int(ged),
        round(fitness_L1W2['perc_fit_traces'], 3),
        round(fitness_L2W1['perc_fit_traces'], 3),
        round(mean([fitness_L1W2['perc_fit_traces'], fitness_L2W1['perc_fit_traces']]), 3)]
results_df.loc[len(results_df)] = res_l

In [ ]:
net1, im1, fm1 = pm4.read_pnml('process_mining/petri_nets/el_fp_NOTmm.pnml')
net2, im2, fm2 = pm4.read_pnml('process_mining/petri_nets/el_sp_NOTmm.pnml')

fitness_L1W2 = pm4.fitness_token_based_replay(el_fp_mm, net2, im2, fm2)
prec_L1W2 = pm4.precision_token_based_replay(el_fp_mm, net2, im2, fm2)
gen_L1W2 = generalization_evaluator.apply(el_fp_mm, net2, im2, fm2)

fitness_L2W1 = pm4.fitness_token_based_replay(el_sp_mm, net1, im1, fm1)
prec_L2W1 = pm4.precision_token_based_replay(el_sp_mm, net1, im1, fm1)
gen_L2W1 = generalization_evaluator.apply(el_sp_mm, net1, im1, fm1)

nx_pn = pm4.convert_petri_net_to_networkx(net1, im1, fm1)
nx_pn2 = pm4.convert_petri_net_to_networkx(net2, im2, fm2)
ged = nx.graph_edit_distance(nx_pn, nx_pn2, timeout=30)

res_l = ['N_Multimorbidity', 'Provider', round(fitness_L1W2['log_fitness'], 3), round(prec_L1W2, 3), round(gen_L1W2, 3),
         round(fitness_L2W1['log_fitness'], 3), round(prec_L2W1, 3), round(gen_L2W1, 3),
         round(mean([fitness_L1W2['log_fitness'], fitness_L2W1['log_fitness']]), 3),
         round(mean([prec_L1W2, prec_L2W1]), 3),
         round(mean([gen_L1W2, gen_L2W1]), 3),
        int(ged),
        round(fitness_L1W2['perc_fit_traces'], 3),
        round(fitness_L2W1['perc_fit_traces'], 3),
        round(mean([fitness_L1W2['perc_fit_traces'], fitness_L2W1['perc_fit_traces']]), 3)]
results_df.loc[len(results_df)] = res_l

In [ ]:
net1, im1, fm1 = pm4.read_pnml('process_mining/petri_nets/el_fa_NOTmm.pnml')
net2, im2, fm2 = pm4.read_pnml('process_mining/petri_nets/el_sa_NOTmm.pnml')

fitness_L1W2 = pm4.fitness_token_based_replay(el_fa_mm, net2, im2, fm2)
prec_L1W2 = pm4.precision_token_based_replay(el_fa_mm, net2, im2, fm2)
gen_L1W2 = generalization_evaluator.apply(el_fa_mm, net2, im2, fm2)

fitness_L2W1 = pm4.fitness_token_based_replay(el_sa_mm, net1, im1, fm1)
prec_L2W1 = pm4.precision_token_based_replay(el_sa_mm, net1, im1, fm1)
gen_L2W1 = generalization_evaluator.apply(el_sa_mm, net1, im1, fm1)

nx_pn = pm4.convert_petri_net_to_networkx(net1, im1, fm1)
nx_pn2 = pm4.convert_petri_net_to_networkx(net2, im2, fm2)
ged = nx.graph_edit_distance(nx_pn, nx_pn2, timeout=30)

res_l = ['N_Multimorbidity', 'Activity', round(fitness_L1W2['log_fitness'], 3), round(prec_L1W2, 3), round(gen_L1W2, 3),
         round(fitness_L2W1['log_fitness'], 3), round(prec_L2W1, 3), round(gen_L2W1, 3),
         round(mean([fitness_L1W2['log_fitness'], fitness_L2W1['log_fitness']]), 3),
         round(mean([prec_L1W2, prec_L2W1]), 3),
         round(mean([gen_L1W2, gen_L2W1]), 3),
        int(ged),
        round(fitness_L1W2['perc_fit_traces'], 3),
        round(fitness_L2W1['perc_fit_traces'], 3),
        round(mean([fitness_L1W2['perc_fit_traces'], fitness_L2W1['perc_fit_traces']]), 3)]
results_df.loc[len(results_df)] = res_l

In [ ]:
net1, im1, fm1 = pm4.read_pnml('process_mining/petri_nets/el_fp_NOTooh.pnml')
net2, im2, fm2 = pm4.read_pnml('process_mining/petri_nets/el_sp_NOTooh.pnml')

fitness_L1W2 = pm4.fitness_token_based_replay(el_fp_ooh, net2, im2, fm2)
prec_L1W2 = pm4.precision_token_based_replay(el_fp_ooh, net2, im2, fm2)
gen_L1W2 = generalization_evaluator.apply(el_fp_ooh, net2, im2, fm2)

fitness_L2W1 = pm4.fitness_token_based_replay(el_sp_ooh, net1, im1, fm1)
prec_L2W1 = pm4.precision_token_based_replay(el_sp_ooh, net1, im1, fm1)
gen_L2W1 = generalization_evaluator.apply(el_sp_ooh, net1, im1, fm1)

nx_pn = pm4.convert_petri_net_to_networkx(net1, im1, fm1)
nx_pn2 = pm4.convert_petri_net_to_networkx(net2, im2, fm2)
ged = nx.graph_edit_distance(nx_pn, nx_pn2, timeout=30)

res_l = ['N_High Out-of-hours care (top 20% overall)', 'Provider', round(fitness_L1W2['log_fitness'], 3), round(prec_L1W2, 3), round(gen_L1W2, 3),
         round(fitness_L2W1['log_fitness'], 3), round(prec_L2W1, 3), round(gen_L2W1, 3),
         round(mean([fitness_L1W2['log_fitness'], fitness_L2W1['log_fitness']]), 3),
         round(mean([prec_L1W2, prec_L2W1]), 3),
         round(mean([gen_L1W2, gen_L2W1]), 3),
        int(ged),
        round(fitness_L1W2['perc_fit_traces'], 3),
        round(fitness_L2W1['perc_fit_traces'], 3),
        round(mean([fitness_L1W2['perc_fit_traces'], fitness_L2W1['perc_fit_traces']]), 3)]
results_df.loc[len(results_df)] = res_l

In [ ]:
net1, im1, fm1 = pm4.read_pnml('process_mining/petri_nets/el_fa_NOTooh.pnml')
net2, im2, fm2 = pm4.read_pnml('process_mining/petri_nets/el_sa_NOTooh.pnml')

fitness_L1W2 = pm4.fitness_token_based_replay(el_fa_ooh, net2, im2, fm2)
prec_L1W2 = pm4.precision_token_based_replay(el_fa_ooh, net2, im2, fm2)
gen_L1W2 = generalization_evaluator.apply(el_fa_ooh, net2, im2, fm2)

fitness_L2W1 = pm4.fitness_token_based_replay(el_sa_ooh, net1, im1, fm1)
prec_L2W1 = pm4.precision_token_based_replay(el_sa_ooh, net1, im1, fm1)
gen_L2W1 = generalization_evaluator.apply(el_sa_ooh, net1, im1, fm1)

nx_pn = pm4.convert_petri_net_to_networkx(net1, im1, fm1)
nx_pn2 = pm4.convert_petri_net_to_networkx(net2, im2, fm2)
ged = nx.graph_edit_distance(nx_pn, nx_pn2, timeout=30)

res_l = ['N_High Out-of-hours care (top 20% overall)', 'Activity', round(fitness_L1W2['log_fitness'], 3), round(prec_L1W2, 3), round(gen_L1W2, 3),
         round(fitness_L2W1['log_fitness'], 3), round(prec_L2W1, 3), round(gen_L2W1, 3),
         round(mean([fitness_L1W2['log_fitness'], fitness_L2W1['log_fitness']]), 3),
         round(mean([prec_L1W2, prec_L2W1]), 3),
         round(mean([gen_L1W2, gen_L2W1]), 3),
        int(ged),
        round(fitness_L1W2['perc_fit_traces'], 3),
        round(fitness_L2W1['perc_fit_traces'], 3),
        round(mean([fitness_L1W2['perc_fit_traces'], fitness_L2W1['perc_fit_traces']]), 3)]
results_df.loc[len(results_df)] = res_l

In [ ]:
results_df

In [ ]:
results_df.to_csv('process_mining/crosslog_cc_results_v2.csv', index=False)